In [18]:
# !pip install category_encoders
# !pip install pycaret[full]

In [19]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from category_encoders import TargetEncoder
from pycaret.classification import load_model, predict_model
from sklearn.metrics import classification_report, confusion_matrix

import os
import warnings
warnings.filterwarnings('ignore')


# Set paths
BASE_DIR = '/content'
MODEL_PATH = os.path.join(BASE_DIR, "no_SMOTE_model_optimal_weights")
INPUT_CSV_PATH = os.path.join(BASE_DIR, "b_test.csv")  # Path to your unseen test data

In [20]:

# Load the saved model
model = load_model(MODEL_PATH)


Transformation Pipeline and Model Successfully Loaded


In [28]:

# Load unseen data
input_df = pd.read_csv(INPUT_CSV_PATH)


In [29]:


# Drop target column if present
if 'is_hr' in input_df.columns:
    input_df = input_df.drop(columns=['is_hr'])

# ===== Preprocessing =====
# Encode categorical features
categorical_cols = ['stand', 'p_throws', 'inning_topbot', 'if_fielding_alignment', 'of_fielding_alignment']
for col in categorical_cols:
    le = LabelEncoder()
    le.fit(input_df[col].astype(str))
    input_df[col] = le.transform(input_df[col].astype(str))

# Target encoding for high-cardinality features (fit with dummy target)
te = TargetEncoder(cols=['home_team', 'pitch_type'])
dummy_target = np.zeros(len(input_df))
input_df = te.fit_transform(input_df, dummy_target)

# Convert boolean columns to int
for col in ['on_3b', 'on_2b', 'on_1b']:
    input_df[col] = input_df[col].astype(int)

# ===== Predict only on first 5 rows =====
subset = input_df.head(5)
predictions = predict_model(model, data=subset)

# Show results
print(predictions[['prediction_label', 'prediction_score']])


   prediction_label  prediction_score
0                 0            1.0000
1                 0            1.0000
2                 0            1.0000
3                 0            0.9998
4                 0            0.9998
